In [ ]:
# | include: false
# | default_exp scilint

In [ ]:
# | export

import ast
import os
import re
from collections import Counter
from pathlib import Path

import nbformat
import numpy as np
import pandas as pd
from execnb.nbio import read_nb
from fastcore.script import call_parse
from nbdev.clean import nbdev_clean
from nbdev.doclinks import nbdev_export, nbglob
from nbdev.test import nbdev_test
from nbqa.__main__ import _get_configs, _main
from nbqa.cmdline import CLIArgs
from nbqa.find_root import find_project_root

In [ ]:
%load_ext autoreload
%autoreload 2

# Read-in Data

In [ ]:
nbdev_path = Path(Path(".").resolve(), "example_nbs", "nbdev.ipynb")
nbdev_hq_path = Path(Path(".").resolve(), "example_nbs", "nbdev_high_quality.ipynb")
non_nbdev_path = Path(Path(".").resolve(), "example_nbs", "non_nbdev.ipynb")
non_nbdev_lq_path = Path(
    Path(".").resolve(), "example_nbs", "non_nbdev_low_quality.ipynb"
)

nbdev_nb = read_nb(nbdev_path)
nbdev_hq_nb = read_nb(nbdev_hq_path)
non_nbdev_nb = read_nb(non_nbdev_path)
non_nbdev_lq_nb = read_nb(non_nbdev_lq_path)

# NB Code Style

In [ ]:
# | export


def run_nbqa_cmd(cmd):
    print(f"Running {cmd}")
    project_root: Path = find_project_root(tuple([str(Path(".").resolve())]))
    args = CLIArgs.parse_args([cmd, str(project_root)])
    configs = _get_configs(args, project_root)
    output_code = _main(args, configs)
    return output_code

In [ ]:
project_root: Path = find_project_root(tuple([str(Path(".").resolve())]))
assert os.path.basename(project_root) == "scilint"

In [ ]:
# | export


@call_parse
def scilint_tidy():
    """
    Run notebook formatting and tidy utilities.
    These tools should be configured to run automatically without intervention."
    """
    tidy_tools = ["black", "isort", "autoflake"]
    [run_nbqa_cmd(c) for c in tidy_tools]

# Quality relevant data extraction

## Definitions
* Function ($f$) = function in `# export` block
* Test ($\tau$) = call of exported function outside `# export` block

## Metrics
1. Tests per Function: $\mathrm{TpF}$ = $\dfrac{|\tau|}{f}$,when $f=0; \mathrm{TpF} = 0$
2. In-function Percentage: $\mathrm{IP} = $$\mathrm{statementsInFunction}:$$\mathrm{allStatements}$ 
3. MD to Code Ratio: $\mathrm{CMR}$ = $ \mathrm{markdownCells}:$$\mathrm{codeCells}$ 
4. Total Code Lines: $\mathrm{TCL}$ = $\mathrm{allCodeLines}$ 

## Helpers

In [ ]:
# | export


def get_function_defs(code, ignore_private_prefix=True):
    func_names = []
    for stmt in ast.walk(ast.parse(code)):
        if isinstance(stmt, ast.FunctionDef):
            inner_cond = (
                False if ignore_private_prefix and stmt.name.startswith("_") else True
            )
            if inner_cond:
                func_names.append(stmt.name)
    return func_names

In [ ]:
# todo test me
# get_function_defs

In [ ]:
# | export


def count_func_calls(code, func_defs):
    func_calls = Counter({k: 0 for k in func_defs})
    for stmt in ast.walk(ast.parse(code)):
        if isinstance(stmt, ast.Call):
            func_name = stmt.func.id if "id" in stmt.func.__dict__ else stmt.func.attr
            if func_name in func_defs:
                if func_name in func_calls:
                    func_calls[func_name] += 1
    return func_calls

In [ ]:
test_code = """self.hierarchical_topic_reduction(3); 
topic_reduction(3); 
lambda x: topic(x); 
hierarchical_topic_reduction[4]; 
hierarchical_topic_reduction(4); 
blabla()
"""
test_func_defs = [
    "topic",
    "topic_reduction",
    "blablabla",
    "hierarchical_topic_reduction",
]

In [ ]:
assert count_func_calls(test_code, test_func_defs) == Counter(
    {
        "topic": 1,
        "topic_reduction": 1,
        "blablabla": 0,
        "hierarchical_topic_reduction": 2,
    }
)

In [ ]:
nb_cell_code = r"""
def something():
    pass; pass # in x 2
    
%load_ext autoreload
%autoreload 2

!ls -l
if 1!= 2:
    print(4)
#| export

import pandas as pd # out
from sciflow.utils import lib_path, odbc_connect, query # out

#| export

def nb_to_sagemaker_pipeline(
    nb_path: Path,
    silent: bool = True,
):
    nb = read_nb(nb_path)  # in
    lib_name = get_config().get("lib_name")  # in
    module_name = find_default_export(nb["cells"])  # in
    
x = [1,2,3] # out
nb_to_sagemaker_pipeline() # out
"""

In [ ]:
# | export


def replace_ipython_magics(code):
    # Replace Ipython magic and shell command symbol with comment
    code = code.replace("%", "#")
    code = re.sub(r"^!", "#", code)
    return re.sub(r"\n\W?!", "\n#", code)

In [ ]:
throws = False
try:
    assert ast.parse(nb_cell_code)
except SyntaxError:
    throws = True
assert throws
assert type(ast.parse(replace_ipython_magics(nb_cell_code))) == ast.Module

In [ ]:
# | export


def safe_div(numer, denom):
    return 0 if denom == 0 else numer / denom

In [ ]:
assert safe_div(1, 1) == 1
assert safe_div(2, 1) == 2
assert safe_div(1, 2) == 0.5
assert safe_div(0, 1) == 0
assert safe_div(1, 0) == 0
assert safe_div(10, 1) == 10

In [ ]:
# | export


def get_cell_code(nb):
    pnb = nbformat.from_dict(nb)
    nb_cell_code = "\n".join(
        [
            replace_ipython_magics(c["source"])
            for c in pnb.cells
            if c["cell_type"] == "code"
        ]
    )
    return nb_cell_code

## 1. Calls-per-Function

In [ ]:
# | export


def calls_per_func(nb):
    nb_cell_code = get_cell_code(nb)
    func_defs = get_function_defs(nb_cell_code)
    func_calls = count_func_calls(nb_cell_code, func_defs)
    return func_calls

In [ ]:
# | export


def mean_cpf(nb):
    return pd.Series(calls_per_func(nb)).mean()

In [ ]:
# | export


def median_cpf(nb):
    return pd.Series(calls_per_func(nb)).median()

In [ ]:
assert mean_cpf(nbdev_nb).round(2) == 2.23
assert median_cpf(nbdev_nb) == 1

In [ ]:
assert mean_cpf(read_nb(nbdev_path)).round(2) == 2.23
assert mean_cpf(read_nb(nbdev_hq_path)).round(2) == 2.5
assert mean_cpf(read_nb(non_nbdev_path)).round(2) == 1.0
assert mean_cpf(read_nb(non_nbdev_lq_path)).round(2) == 1.62

In [ ]:
assert median_cpf(read_nb(nbdev_path)) == 1.0
assert median_cpf(read_nb(nbdev_hq_path)).round(2) == 1.5
assert median_cpf(read_nb(non_nbdev_path)).round(2) == 1.0
assert median_cpf(read_nb(non_nbdev_lq_path)).round(2) == 1.0

## 2. Asserts-to-Function Ratio

In [ ]:
asserted_code = r"""
def something():
    pass; pass # in x 2
    
assert True

#| export

def nb_to_sagemaker_pipeline(
    nb_path: Path,
    silent: bool = True,
):
    nb = read_nb(nb_path)  # in
    lib_name = get_config().get("lib_name")  # in
    module_name = find_default_export(nb["cells"])  # in
    
x = [1,2,3] # out
assert len(x) > 2
assert something() is None # something +1

def tr():
    return True
    
def get_seg(num):
    return 2
    
assert(tr)
assert(tr()) # tr +1
assert(tr() == 4) # tr +1
assert(4 ==tr()) # tr +1
assert 0 != 0
assert "' '".join(tr(1)) == "00" # tr +1
assert len(get_seg(50)) == 50 # get_seg +1
assert max([int(x) for x in get_seg(100)]) == 99 # get_seg +1
"""

In [ ]:
import nbformat as nbf

asserted_nb = nbf.v4.new_notebook()
asserted_nb["cells"] = [nbf.v4.new_code_cell(asserted_code)]

In [ ]:
# | export


def afr(nb):
    nb_cell_code = get_cell_code(nb)
    func_defs = get_function_defs(nb_cell_code)
    num_funcs = len(func_defs)

    assert_count = 0
    for stmt in ast.walk(ast.parse(nb_cell_code)):
        if isinstance(stmt, ast.Assert):
            assert_count += 1

    return safe_div(assert_count, num_funcs)

In [ ]:
afr(nbdev_nb)

1.3076923076923077

In [ ]:
afr(nbdev_hq_nb)

1.6666666666666667

In [ ]:
afr(non_nbdev_nb)

0.0

In [ ]:
afr(non_nbdev_lq_nb)

0.0

## 3. In-line Asserts Per Function

In [ ]:
# | export


def count_inline_asserts(code, func_defs):
    inline_func_asserts = Counter({k: 0 for k in func_defs})

    for stmt in ast.walk(ast.parse(code)):
        if isinstance(stmt, ast.Assert):
            for assert_st in ast.walk(stmt):
                if isinstance(assert_st, ast.Call):
                    func_name = (
                        assert_st.func.id
                        if "id" in assert_st.func.__dict__
                        else assert_st.func.attr
                    )
                    if func_name in inline_func_asserts:
                        inline_func_asserts[func_name] += 1
    return inline_func_asserts

In [ ]:
# | export


def iaf(nb):
    nb_cell_code = get_cell_code(nb)
    func_defs = get_function_defs(nb_cell_code)
    return count_inline_asserts(nb_cell_code, func_defs)

In [ ]:
func_defs = get_function_defs(asserted_code)
inline_asserts_expected = Counter(
    {"something": 1, "tr": 4, "get_seg": 2, "nb_to_sagemaker_pipeline": 0}
)
inline_asserts_actual = count_inline_asserts(asserted_code, func_defs)

In [ ]:
assert inline_asserts_actual == inline_asserts_expected

In [ ]:
assert 0.0 == pd.Series(iaf(nbdev_nb)).median()
assert 0.0 == pd.Series(iaf(nbdev_hq_nb)).median()
assert 0.0 == pd.Series(iaf(non_nbdev_nb)).median()
assert 0.0 == pd.Series(iaf(non_nbdev_lq_nb)).median()

In [ ]:
iaf(non_nbdev_nb)

Counter({'scalar': 0, 'py_advanced': 0, 'pandas': 0})

In [ ]:
iaf(non_nbdev_lq_nb)

Counter({'get_traffic_text': 0,
         'get_experiment_segment': 0,
         'evaluate': 0,
         'serve_num_topics': 0,
         'get_num_topics': 0,
         'get_topic_sizes': 0,
         'get_topics': 0,
         'plot_wordcloud': 0})

In [ ]:
assert inline_asserts_expected == iaf(asserted_nb)

In [ ]:
# | export


def mean_iaf(nb):
    return pd.Series(iaf(nb)).mean()

In [ ]:
# | export


def median_iaf(nb):
    return pd.Series(iaf(nb)).median()

## Full Code Coverage?

How does pytest-cov do it?

## 2. In-function Percentage

In [ ]:
# | export


def calc_ifp(nb_cell_code):
    stmts_in_func = 0
    stmts_outside_func = 0
    for stmt in ast.walk(ast.parse(replace_ipython_magics(nb_cell_code))):
        if isinstance(stmt, ast.FunctionDef) and not stmt.name.startswith("_"):
            for body_item in stmt.body:
                stmts_in_func += 1
        elif isinstance(stmt, ast.Module):
            for body_item in stmt.body:
                if not isinstance(body_item, ast.FunctionDef):
                    stmts_outside_func += 1
    return (
        0
        if stmts_outside_func + stmts_in_func == 0
        else (stmts_in_func / (stmts_outside_func + stmts_in_func)) * 100
    )

In [ ]:
assert (calc_ifp(nb_cell_code)) == (5 / (5 + 5)) * 100

In [ ]:
# | export


def ifp(nb):
    nb_cell_code = "\n".join(
        [
            replace_ipython_magics(c["source"])
            for c in nb.cells
            if c["cell_type"] == "code"
        ]
    )
    return calc_ifp(nb_cell_code)

In [ ]:
assert ifp(nbdev_nb) >= 0
assert ifp(nbdev_hq_nb) >= 0
assert ifp(non_nbdev_nb) >= 0
assert ifp(non_nbdev_lq_nb) >= 0

## 3. Markdown to Code Percent

In [ ]:
# | export


def mcp(nb):
    md_cells = [c for c in nb.cells if c["cell_type"] == "markdown"]
    code_cells = [c for c in nb.cells if c["cell_type"] == "code"]
    num_code_cells = len(code_cells)
    num_md_cells = len(md_cells)
    return (
        0
        if num_code_cells == 0
        else (num_md_cells / (num_md_cells + num_code_cells)) * 100
    )

In [ ]:
assert mcp(nbdev_nb) >= 0
assert mcp(nbdev_hq_nb) >= 0
assert mcp(non_nbdev_nb) >= 0
assert mcp(non_nbdev_lq_nb) >= 0

## 4. Total Code Length

In [ ]:
# | export


def tcl(nb):
    return sum([len(c["source"]) for c in nb.cells if c["cell_type"] == "code"])

In [ ]:
assert tcl(nbdev_nb) >= 50
assert tcl(nbdev_hq_nb) >= 50
assert tcl(non_nbdev_nb) >= 50
assert tcl(non_nbdev_lq_nb) >= 50

In [ ]:
# | export


def lint_nb(
    nb_path,
    tpf_warn_thresh=None,
    ifp_warn_thresh=None,
    afr_warn_thresh=1,
    iaf_med_warn_thresh=0,
    iaf_mean_warn_thresh=0.5,
    mcp_warn_thresh=None,
    tcl_warn_thresh=None,
    rounding_precision=3,
):
    nb = read_nb(nb_path)
    (np.nan, np.nan, np.nan, np.nan)
    nb_cpf_median = round(median_cpf(nb), rounding_precision)
    nb_cpf_mean = round(mean_cpf(nb), rounding_precision)
    nb_ifp = round(ifp(nb), rounding_precision)
    nb_afr = round(afr(nb), rounding_precision)
    nb_iaf_median = round(median_iaf(nb), rounding_precision)
    nb_iaf_mean = round(mean_iaf(nb), rounding_precision)
    nb_mcp = round(mcp(nb), rounding_precision)
    nb_tcl = round(tcl(nb), rounding_precision)
    # print(f"NB: {nb_path.name} CallsPerFunction (Median): {nb_cpf_median} CallsPerFunction (Mean): {nb_cpf_mean} \
    # In-FunctionPercent: {nb_ifp} AssertsPerFunction: {nb_cpf_median} InlineAssertsPerFunction (Median): {nb_iaf_median} \
    # InlineAssertsPerFunction (Mean): {nb_iaf_mean} MarkdownToCodeRatio: {nb_mcp} TotalCodeLen: {nb_tcl}")
    return (
        nb_cpf_median,
        nb_cpf_mean,
        nb_ifp,
        nb_afr,
        nb_iaf_median,
        nb_iaf_mean,
        nb_mcp,
        nb_tcl,
    )

In [ ]:
# | export


def format_quality_warning(metric, warning_data, warn_thresh, direction):
    for warning_row in warning_data.reset_index().itertuples():
        print(f'"{warning_row.index}" has: {metric} {direction} {warn_thresh}')

In [ ]:
# | export


def lint_nbs(
    cpf_med_warn_thresh=1,
    cpf_mean_warn_thresh=1,
    ifp_warn_thresh=20,
    afr_warn_thresh=1,
    iaf_med_warn_thresh=0,
    iaf_mean_warn_thresh=0.5,
    mcp_warn_thresh=5,
    tcl_warn_thresh=20000,
    rounding_precision=3,
    csv_out_path="/tmp/lint.csv",
):
    nb_paths = [Path(p) for p in nbglob()]
    lt_metric_cols = [
        "calls_per_function_median",
        "calls_per_function_mean",
        "in_function_pct",
        "asserts_function_ratio",
        "inline_asserts_per_function_median",
        "inline_asserts_per_function_mean",
        "markdown_code_pct",
    ]
    gt_metric_cols = ["total_code_len"]
    lt_metrics_thresholds = [
        cpf_med_warn_thresh,
        cpf_mean_warn_thresh,
        ifp_warn_thresh,
        afr_warn_thresh,
        iaf_med_warn_thresh,
        iaf_mean_warn_thresh,
        mcp_warn_thresh,
    ]
    gt_metrics_thresholds = [tcl_warn_thresh]
    results = []
    nb_names = []
    for nb_path in nb_paths:
        nb_names.append(nb_path.stem)
        results.append(lint_nb(nb_path))
    lint_report = pd.DataFrame.from_records(
        data=results, index=nb_names, columns=lt_metric_cols + gt_metric_cols
    ).sort_values(["calls_per_function_median", "markdown_code_pct"], ascending=False)

    # TODO persist to remote storage
    # needs to be tied to a flow execution rather than a build
    # what is the best way to do this?

    print("\n*********************Begin Scilint Report*********************")
    issues_raised = False
    for lt_metric_col, lt_metrics_threshold in zip(
        lt_metric_cols, lt_metrics_thresholds
    ):
        metrics_series = lint_report[lt_metric_col]
        warning_data = metrics_series[metrics_series < lt_metrics_threshold]
        if len(warning_data) > 0:
            issues_raised = True
        format_quality_warning(
            lt_metric_col,
            warning_data,
            lt_metrics_threshold,
            direction="<",
        )
    for gt_metric_col, gt_metrics_threshold in zip(
        gt_metric_cols, gt_metrics_thresholds
    ):
        metrics_series = lint_report[gt_metric_col]
        warning_data = metrics_series[metrics_series > gt_metrics_threshold]
        if len(warning_data) > 0:
            issues_raised = True
        format_quality_warning(
            gt_metric_col,
            warning_data,
            gt_metrics_threshold,
            direction=">",
        )
    if not issues_raised:
        print("No issues found")
    print("*********************End Scilint Report***********************")

    lint_report.to_csv(csv_out_path)

    return lint_report

In [ ]:
lint_report = lint_nbs()


*********************Begin Scilint Report*********************
"index" has: in_function_pct < 20
"non_nbdev_low_quality" has: asserts_function_ratio < 1
"non_nbdev" has: asserts_function_ratio < 1
"index" has: asserts_function_ratio < 1
"non_nbdev_low_quality" has: inline_asserts_per_function_mean < 0.5
"non_nbdev" has: inline_asserts_per_function_mean < 0.5
"non_nbdev" has: markdown_code_pct < 5
*********************End Scilint Report***********************


In [ ]:
lint_report

,calls_per_function_median,calls_per_function_mean,in_function_pct,asserts_function_ratio,inline_asserts_per_function_median,inline_asserts_per_function_mean,markdown_code_pct,total_code_len
scilint,2.5,3.250,52.023,1.667,0.0,1.500,17.647,14275
nbdev_high_quality,1.5,2.500,44.118,1.667,0.0,1.000,30.769,4978
nbdev,1.0,2.231,50.725,1.308,0.0,0.846,30.769,4918
non_nbdev_low_quality,1.0,1.625,45.000,0.000,0.0,0.000,11.111,2955
non_nbdev,1.0,1.000,35.714,0.000,0.0,0.000,0.000,1233
index,NaN,NaN,0.000,0.000,NaN,NaN,77.778,8


In [ ]:
# | export


@call_parse
def scilint_lint():
    lint_nbs()

In [ ]:
# | export


@call_parse
def scilint_build():
    nbdev_export()
    nbdev_test()
    scilint_tidy()
    scilint_lint()
    nbdev_clean()